In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import statsmodels.stats.api as sms
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.tsaplots import plot_acf
import warnings
import csv
import json
import string
import re
from datetime import datetime

In [17]:
year_ranges = ['2020-2022', '2022-2023', '2023-2024', '2024-2025']
combined = 'all_combined/Combined_data.csv'

In [3]:
descriptors = ['- single mix', '- remix', '- live', '- extended', '- radio edit', '- mono']

def normalize(name):
    return name.translate(str.maketrans('', '', string.punctuation)).strip().lower()
    
def trim_descriptors(name):
    name = re.sub(r'\s*-\s*feat.*$', '', name, flags=re.IGNORECASE).strip()
    name = re.sub(r'[$$$\<].*?[$$$\>].*$', '', name, flags=re.IGNORECASE).strip()
    for descriptor in descriptors:
        name = name.lower().replace(descriptor, '').strip()
    return name

vect_norm = np.vectorize(normalize)
vect_trim = np.vectorize(trim_descriptors)

In [7]:
song_age = 'song_age/song_age_data'+year_ranges[0]+'.csv'
df_combined = pd.read_csv(song_age)

In [9]:
for k in range(1,len(year_ranges)): #replace other num with: len(year_ranges)
    found_index = []
    df_combined = df_combined.reset_index() 
    df_combined = df_combined.drop(["index"], axis = 1)
    a = df_combined["Track Name"].to_numpy()
    a = vect_norm(a)
    
    song_age = 'song_age/song_age_data'+year_ranges[k]+'.csv'
    df_2 = pd.read_csv(song_age)
    b = df_2["Track Name"].to_numpy()
    b = vect_norm(b)

    for song in b:
        found_b = np.where(b == song)
        
        if song in a: # found a match, add play_count data
            found_a = np.where(a == song)
            if found_a[0][0] not in found_index:
                #print(df_2.loc[found_b[0][0]]['Track Name'])
                found_index.append(found_a[0][0])
                df_combined.loc[found_a[0][0], 'Play Count'] += df_2.loc[found_b[0][0], 'Play Count']
            #print("a", found_a[0])
            #for i in range(1,len(found_a[0])):
            #    df_combined = df_combined.reset_index() 
            #    df_combined = df_combined.drop(["index"], axis = 1)
            #    df_combined = df_combined.drop(index=[found_a[0][-i]], axis = 1)
            #a = df_combined["Track Name"].to_numpy()
            #a = vect_norm(a)
            #found_a = np.where(a == song)
            #print("a after", found_a[0])
            #df_combined.loc[found_a[0][0], 'Play Count'] += df_2.loc[found_b[0][0], 'Play Count']
            
            
        else: # new song found, append to the end
            #print(df_2.iloc[found_b[0][0]]["Track Name"])
            df_combined.loc[df_combined.shape[0]+1] = df_2.iloc[found_b[0][0]]
            #print("b", found_b[0])
            #for i in range(1,len(found_b[0])):
            #    df_2 = df_2.reset_index() 
            #    df_2 = df_2.drop(["index"], axis = 1)
            #    df_2 = df_2.drop(index=[found_b[0][-i]], axis = 1)
            #b = df_2["Track Name"].to_numpy()
            #b = vect_norm(b)
            #found_b = np.where(b == song)
            #print("b after", found_b[0])
            

In [11]:
df_combined.to_csv(combined, index=False)